#### Conventional Event Handling : where each event has a corresponding event-handling class

- Four seperate handler classses
- we throw away unhandled events - we could have passed None or Nothing as MouseHandler() argument
- order in which we create them does not matter since each handles events only of the type it designed for

The chain-of-responsibility pattern is structurally nearly identical to the decorator pattern, the difference being that for the decorator, all classes handle the request, while for the chain of responsibility, exactly one of the classes in the chain handles the request.

In [11]:
# sys module - This module provides access to some variables used or maintained by the interpreter and to functions that interact strongly with the interpreter. It is always available.
# Pre-requesitie: pip install events (Python 3). an event is a “slot” where callback functions (event handlers) can be attached to - a process referred to as subscribing to an event.

import sys
from events import Events

In [19]:
def main():
    print ("Handler Chain #1")
    handler1 = TimerHandler(KeyHandler(MouseHandler(NullHandler())))
    # this handler must be first in chain since it is used to eavesdrop on the events passing into the chainand to report them
    # but not to handle them
    handler2 = DebugHandler(handler1)
    # we can pass None or nothing instead of null handler
    # exit the loop and terminate the application if the event is TERMINATE
    while True:
        event = Events.next()
        if event.kind == Events.TERMINATE:
            break
        handler1.handle(event)
        handler2.handle(event) # for debugging purposes

In [ ]:
class NullHandler():
    
    """
        a) this class serves as a base class
    """
    
    def __init__(self, successor=None):
        """ 
            Initializing a class attribute to None
            __double_leading_underscore: when naming a class attribute, 
            invokes name mangling (inside class FooBar, __boo becomes _FooBar__boo; see below).
            
            __double_leading_underscore is still public, the variable is simply renamed to avoid a clash
        """
        self.__successor = successor
    
    def handle(self,event):
        # if we have not chose to discard the request, then we will pass to the successor in the chain
        # if you want you can log or raise an error
        if self.__successor is not None:
            self.__successor.handle(event)

class DebugHandler(NullHandler):
    
    """
        a) different from other handlers
        b) it never handles any events and it must be first in the chain
        c) it takes a file or file-like object to direct its report to,and when an event occurs, it reports the event and 
           passes it on
    """
    
    def __init__(self, successor = None, file = sys.stdout):
        # short cut to initialize parent class along with the current subclass
        super().__init__(successor)
        self.__file = file
    
    def handle(self, event):
        self.__file.write("*DEBUG*:{}\n".format(event))
        # Super can be called upon in a single inheritance, in order to refer to the parent class or multiple classes without explicitly naming them. It’s somewhat of a shortcut, but more importantly, it helps keep your code maintainable for the foreseeable future.
        super().handle(event)

class MouseHandler(NullHandler):
    
    """
        note that __init__ is not re-implemented. self.__successor variable will be correctly created
        this handler class handles only those events that it is interested in MOUSE and passes any other events in the chain
    """
    def handle(self, event):
        if event.kind == Events.MOUSE:
            print("Click:{}".format(event))
        else:
            super().handle(event)
            
class KeyHandler(NullHandler):
    
    def handle(self, event):
        if event.kind == Events.KEYPRESS:
            print("Press: {}".format(event))
        else:
            super().handle(event)

class TimerHandler(NullHandler):
    
    def handle(self, event):
        if event.kind == Events.TIMER:
            print ("Timeout:{}".format(event))
        else:
            super.handle(event)

if __name__ == "__main__":
    main()